In [0]:
# Load data - Explore the smaller October dataset
df_oct = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)

Step 1: Write df_oct as Delta

df_oct.write
→ Starts a write operation from a Spark DataFrame

.format("delta")
→ Tells Spark to use Delta Lake, not Parquet or CSV

.mode("overwrite")
→ Replace existing data if it exists

.save("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")
→ Saves data to storage
→ This folder will now contain:

Parquet files and
_delta_log/ 

In [0]:
%sql
SHOW VOLUMES IN workspace.ecommerce;

database,volume_name
ecommerce,ecommerce_data


In [0]:
df_oct.write \
    .format("delta") \
    .mode("overwrite") \
    .save("/Volumes/workspace/ecommerce/ecommerce_data/df_oct_delta")


The above cell just writes files to a location as Delta files, not a named table. We just saved files to disk. We did not create a table that the SQL engine knows about.

To read this, we need to manually specify the path as there's no name that Spark remembers.

In [0]:
spark.read.format("delta").load("/Volumes/workspace/ecommerce/ecommerce_data/df_oct_delta")


DataFrame[event_time: timestamp, event_type: string, product_id: int, category_id: bigint, category_code: string, brand: string, price: double, user_id: int, user_session: string]

Delta data already exists on disk
Now we are just registering it as a table.

There are two ways to create Delta tables:

[1] External Delta table → data already exists at a path
[2] Managed Delta table → Databricks manages storage

In [0]:
# Create managed table
df_oct.write.format("delta").saveAsTable("df_oct_table")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7293070745601141>, line 2
      1 # Create managed table
----> 2 df_oct.write.format("delta").saveAsTable("df_oct_table")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1554     req.user_context.user_id = self._user_id
   1555 req.plan.command.CopyFrom(c

Here, in the above cell, we register the table name. The name of the table is df_oct_table. Now Spark remembers where the data resides. We can read it using SQL or Python as shown below:

SELECT COUNT(*) FROM df_oct_table

spark.table("df_oct_table")

There’s no path needed — Spark already knows where the data is stored.

Schema enforcement - small example

In [0]:
# Create a small Delta table that has strict schema
schema_sample = "user_id INT, price DOUBLE, purchase_mode STRING"

data_sample = [(100, 100.0, "store"), (200, 250.5, "online")]

df_sample = spark.createDataFrame(data_sample, schema=schema_sample)

df_sample.write.format("delta").mode("overwrite").saveAsTable("delta_schema_test")


In [0]:
# Insert incompatible data

# Wrong schema – price is string, user_id missing
flawed_data = [("Shah Rukh Khan", "100", "purchase")]

df_flawed = spark.createDataFrame(flawed_data, ["user_id", "price", "purchase_mode"])

df_flawed.write.format("delta").mode("append").saveAsTable("delta_schema_test")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6976869407386029>, line 8
      4 flawed_data = [("Shah Rukh Khan", "100", "purchase")]
      6 df_flawed = spark.createDataFrame(flawed_data, ["user_id", "price", "purchase_mode"])
----> 8 df_flawed.write.format("delta").mode("append").saveAsTable("delta_schema_test")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_c

In [0]:
# Tect schema enforcement on df_oct

In [0]:
df_oct.printSchema()


root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
# Test schema enforcement using df_oct
try:
    wrong_schema = spark.createDataFrame([("Ullu","Goobe","Owl")], ["event_time","event_type","product_id"])
    wrong_schema.write.format("delta").mode("append").saveAsTable("df_oct_table")
except Exception as e:
    print(f"Schema enforcement: {e}")

Schema enforcement: [DELTA_FAILED_TO_MERGE_FIELDS] Failed to merge fields 'event_time' and 'event_time'.

JVM stacktrace:
com.databricks.sql.transaction.tahoe.DeltaAnalysisException
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.$anonfun$mergeDataTypes$1(SchemaMergingUtils.scala:231)
	at scala.collection.ArrayOps$.map$extension(ArrayOps.scala:936)
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.merge$1(SchemaMergingUtils.scala:217)
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.mergeDataTypes(SchemaMergingUtils.scala:335)
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.mergeSchemas(SchemaMergingUtils.scala:179)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation$.mergeSchema(ImplicitMetadataOperation.scala:359)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation.updateMetadata(ImplicitMetadataOperation.scala:112)
	at com.databricks.sql.transaction.tahoe.schema.I